In [1]:
import os
import astropy
import numpy as np
from astropy.table import Table
from astropy.table import Column
import glob
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from mpl_toolkits.mplot3d import Axes3D

In [2]:
sample = Table.read("/home/xhall/Documents/NewZTF/ML_sample.ascii", format = "ascii")
sample.rename_column('col1', 'ZTF_Name')
sample.rename_column('col2', "Class")
sample.rename_column('col8', "Version")

In [3]:
sample_2018 = Table.from_pandas(pd.read_hdf("/home/xhall/Documents/NewZTF/final_rcf_table.h5"))

In [4]:
joined_sample = astropy.table.join(sample_2018,sample)

In [5]:
sample[0]

ZTF_Name,Class,col3,col4,col5,col6,col7,Version,col9
str12,str12,float64,float64,float64,float64,float64,str43,int64
ZTF18aabssth,II,0.026,0.022914,2458186.5,2458218.66,-32.16,ZTF18aabssth_20180309_P60_v1.ascii,1


In [6]:
np.size(np.unique(sample["ZTF_Name"]))

4411

In [12]:
snidoutput = np.load("//home/xhall/Documents/NewZTF/sample_2018/SNID_2018_output.npy",allow_pickle=True)

In [13]:
snidoutput[0][0]

'ZTF18aabssth_20180309_P60_v1'

ResultsTable_SNID = Table(
                    names=("Version", "c_rlap", "c_snid"
                    ),
                    meta={"name": "Class Results"},
                    dtype=("U64", "float64", "U64"
                          )
                    )
for j in snidoutput:
    row = []
    row.append(j[0] + ".ascii")
    row.append(j[1][0]["rlap"])
    row.append(j[1][0]["type"])
    ResultsTable_SNID.add_row(row)

In [14]:
ZTable_best = Table(
                    names=("Version", "ZTF_Name", "sntemplate", "c_rlap", "c_snid", "z_rlap", "z_snid", "z_snid_err", "age", "age_flag", "z_level"
                    ),
                    meta={"name": "Redscore Results"},
                    dtype=("U64", "U64", "U64", "float64", "U64", "float64", "float64","float64", "float64", "int32", "int32"
                          )
                    )
for j in snidoutput:
    row = []
    row.append(j[0] + ".ascii")
    row.append(j[0].split("_")[0])
    row.append(j[1][0]["sn"])
    row.append(j[1][0]["rlap"])
    row.append(j[1][0]["type"])
    good = j[1][np.where(j[1]["grade"] == "good")]
    if("SLSN" in str(j[1][0]["type"])):
        good = good[np.where(good["z"] <= .5)]
    else:
        good = good[np.where(good["z"] <= .2)]
    if(len(good) != 0):
        row.append(good[0]["rlap"])
        row.append(float(good[0]["z"]))
        row.append(float(good[0]["zerr"]))
        row.append(float(good[0]["age"]))
        row.append(float(good[0]["age_flag"]))
        row.append(1)
    else:
        row.append(j[1][0]["rlap"])
        row.append(float(j[1][0]["z"]))
        row.append(float(j[1][0]["zerr"]))
        row.append(float(j[1][0]["age"]))
        row.append(float(j[1][0]["age_flag"]))
        row.append(0)
    ZTable_best.add_row(row)

In [15]:
RedshiftClass = astropy.table.join(ZTable_best, joined_sample)

In [16]:
len(RedshiftClass)

1543

In [20]:
RedshiftClass[np.where(RedshiftClass["Version"] == "ZTF18aaxdrjn_20180531_P60_v1.ascii")]

Version,ZTF_Name,sntemplate,c_rlap,c_snid,z_rlap,z_snid,z_snid_err,age,age_flag,z_level,TNS_Name,discovered_by,tns_internal_name,RA_sn,Dec_sn,sn_type,z_sn,g_jd_max,g_max,g_unc_max,r_jd_max,r_max,r_unc_max,ebv,host_name_PS1,RA_host_PS1,Dec_host_PS1,z_host,sep,projected_kpc,gKron_PS1,rKron_PS1,iKron_PS1,zKron_PS1,yKron_PS1,w1_host,w2_host,ebv_host,Class,col3,col4,col5,col6,col7,col9
str64,str64,str64,float64,str64,float64,float64,float64,float64,int32,int32,str9,str13,str29,float64,float64,str9,float64,float64,float64,float64,float64,float64,float64,float64,str21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str12,float64,float64,float64,float64,float64,int64
ZTF18aaxdrjn_20180531_P60_v1.ascii,ZTF18aaxdrjn,sn2007uy,8.82,Ib-norm,8.82,0.0395,0.0084,-6.18,0,1,SN2018cdt,ZTF,ZTF18aaxdrjn,185.00686948460054,56.158869509170785,Ia,0.032,2458279.6963542,16.928712844848633,0.01022690162062645,2458279.6771296,17.034038543701172,0.01343842875212431,0.01841,PSO J185.0063+56.1629,185.00629234,56.16290614,0.033987,14.577857706313072,9.872598716847808,15.097900390625,14.307999610900879,13.927800178527832,13.898799896240234,13.651599884033203,13.73404554575454,14.381776271826267,0.01846,Ia,0.032,0.033987,2458269.5,2458279.65,-10.15,1


In [17]:
np.unique(RedshiftClass["c_snid"])

AGN
C-star
Gal
II-norm
IIb
IIn
Ia-02cx
Ia-03fg
Ia-91T
Ia-91bg
Ia-csm


In [19]:
RedshiftClass.to_pandas().to_csv("/home/xhall/Documents/NewZTF/sample_2018/ML_sample_snid_2018.csv", index=False)

In [ ]:
RedshiftClass = Table.from_pandas(pd.read_csv("/home/xhall/Documents/NewZTF/ML_sample_snid.csv"))

In [ ]:
len(np.unique(RedshiftClass[np.where(RedshiftClass["z_snid"] < .2)]["ZTF_Name"]))

In [ ]:
len(np.unique(RedshiftClass["ZTF_Name"]))

In [59]:
RedshiftClass[np.where(RedshiftClass["c_snid"] == RedshiftClass["Class"])]

Version,ZTF_Name,sntemplate,c_rlap,c_snid,z_rlap,z_snid,z_snid_err,age,age_flag,z_level,Class,col3,col4,col5,col6,col7,col9
str64,str64,str64,float64,str64,float64,float64,float64,float64,int32,int32,str12,float64,float64,float64,float64,float64,int64
ZTF18aabilqu_20180522_P200_v1.ascii,ZTF18aabilqu,agn,15.45,AGN,15.45,0.0219,0.0017,-99.9,2,1,AGN,0.0222,nan,2458260.5,2458598.61,-338.11,0
ZTF18aadkjmo_20181102_P60_v1.ascii,ZTF18aadkjmo,agn,12.57,AGN,12.57,0.0231,0.0027,-99.9,2,1,AGN,0.02499,nan,2458424.5,2458455.72,-31.22,0
ZTF18aahatvc_20200527_P200_v1.ascii,ZTF18aahatvc,sn97br,8.73,Ia-91T,8.73,0.0785,0.0064,12.4,0,1,Ia-91T,0.0778,nan,2458996.5,2458985.72,10.78,0
ZTF18aahatvc_20200527_P200_v4.ascii,ZTF18aahatvc,sn97br,8.73,Ia-91T,8.73,0.0785,0.0064,12.4,0,1,Ia-91T,0.0778,nan,2458996.5,2458985.72,10.78,0
ZTF18aahatvc_20200527_P200_v5.ascii,ZTF18aahatvc,sn97br,8.73,Ia-91T,8.73,0.0785,0.0064,12.4,0,1,Ia-91T,0.0778,nan,2458996.5,2458985.72,10.78,0
ZTF18aahfgyz_20180410_P200_v1.ascii,ZTF18aahfgyz,sn06em,19.62,Ia-91bg,19.62,0.0113,0.0028,4.26,0,1,Ia-91bg,0.011,0.012012,2458218.5,2458218.75,-0.25,1
ZTF18aahhenr_20180411_APO_v1.ascii,ZTF18aahhenr,sn1999aa,9.92,Ia-91T,9.92,0.081,0.0057,-4.1,0,1,Ia-91T,0.078,0.076358,2458219.5,2458222.73,-3.23,1
ZTF18aahhenr_20180412_LT_v1.ascii,ZTF18aahhenr,sn2005hj,6.96,Ia-91T,6.96,0.0784,0.0059,-4.3,0,1,Ia-91T,0.078,0.076358,2458220.5,2458222.73,-2.23,1


In [78]:
RedshiftClass[RedshiftClass["c_snid"] == RedshiftClass["Class"]]

Version,ZTF_Name,sntemplate,c_rlap,c_snid,z_rlap,z_snid,z_snid_err,age,age_flag,z_level,Class,col3,col4,col5,col6,col7,col9
str64,str64,str64,float64,str64,float64,float64,float64,float64,int32,int32,str12,float64,float64,float64,float64,float64,int64
ZTF18aahfgyz_20180410_P200_v1.ascii,ZTF18aahfgyz,sn06em,19.62,Ia-91bg,19.62,0.0113,0.0028,4.26,0,1,Ia-91bg,0.011,0.012012,2458218.5,2458218.75,-0.25,1
ZTF18aajtlbf_20180420_P200_v1.ascii,ZTF18aajtlbf,sn02cf,19.42,Ia-91bg,19.42,0.0301,0.0026,-0.65,0,1,Ia-91bg,0.028,0.027989,2458228.5,2458229.9,-1.4,1
ZTF18aakaljn_20180421_Ekar_v1.ascii,ZTF18aakaljn,sn91T,14.4,Ia-91T,14.4,0.0665,0.0039,-0.2,0,1,Ia-91T,0.066,0.070028,2458229.5,2458227.61,1.89,1
ZTF18aakrnvd_20180517_P200_v1.ascii,ZTF18aakrnvd,sn1998S,10.45,IIn,10.45,0.0446,0.0041,75.5,0,1,IIn,0.047,0.045349,2458255.5,2458312.64,-57.14,1
ZTF18aamftst_20180615_P60_v1.ascii,ZTF18aamftst,sn1997cy,16.83,IIn,16.83,0.0458,0.0038,14.1,1,1,IIn,0.046,0.037801,2458284.5,2458312.64,-28.14,1
ZTF18aaqjovh_20180509_P60_v1.ascii,ZTF18aaqjovh,sn1997ef,6.11,Ic-BL,6.11,0.3352,0.0181,-13.9,0,0,Ic-BL,0.053,0.054031,2458247.5,2458248.69,-1.19,1
ZTF18aasycpd_20180602_P60_v1.ascii,ZTF18aasycpd,sn2000H,9.6,IIb,9.6,0.0499,0.0057,13.4,0,1,IIb,0.05,0.045154,2458271.5,2458270.75,0.75,1
ZTF18abclfee_20180704_LT_v1.ascii,ZTF18abclfee,sn2005cc,12.7,Ia-02cx,12.7,0.0278,0.0023,3.6,0,1,Ia-02cx,0.029,nan,2458303.5,2458299.65,3.85,1
ZTF18abclfee_20180708_P200_v2.ascii,ZTF18abclfee,sn02cx,10.49,Ia-02cx,10.49,0.0301,0.0033,10.3,0,1,Ia-02cx,0.029,nan,2458307.5,2458299.65,7.85,1
